In [1]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

env = make_vec_env(
    "HalfCheetah-v4",
    rng=np.random.default_rng(),
    n_envs=4,
    env_make_kwargs={"render_mode": "rgb_array"},
)


expert = load_policy(
    "sac-huggingface",
    organization="sb3",
    env_name="HalfCheetah-v3",
    venv=env,
)

obs = env.reset()
print("num_env", env.num_envs)
print("num observation", env.observation_space.shape[0])
print("num action", env.action_space.shape[0])

C:\Users\xavier\anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: 'bytes' object cannot be interpreted as an integer
  warnings.warn(
C:\Users\xavier\anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend transitioning to Gymnasium by saving that model again.
  warnings.warn(


num_env 4
num observation 17
num action 6


In [2]:
from src.models.dagger import BC
from src.models.dagger import SimpleDAggerTrainer

In [3]:
bc_trainer = BC(
    observation_space=env.observation_space, 
    action_space=env.action_space, 
    rng=np.random.default_rng())

observation_space Box(-inf, inf, (17,), float64)


In [4]:
dagger_trainer = SimpleDAggerTrainer(
    venv=env,
    expert_policy=expert,
    bc_trainer=bc_trainer,
    rng=np.random.default_rng(),
)

In [5]:
dagger_trainer.train(1000)

round:  0
partial_trajectories {0: [], 1: [], 2: [], 3: []}
Start generate Traj
finish_trajectory
finish_trajectory
finish_trajectory
finish_trajectory
active env sum  0
active env [False False False False]
END
Epoch  0


Training :  15%|██████████▋                                                           | 19/125 [00:15<01:02,  1.71it/s]

loss tensor(7.3753, grad_fn=<AddBackward0>)


Training :  95%|█████████████████████████████████████████████████████████████████▋   | 119/125 [00:16<00:00, 27.55it/s]

loss tensor(5.3706, grad_fn=<AddBackward0>)


...

Training :   6%|████▌                                                                  | 8/125 [00:14<02:38,  1.35s/it]

loss tensor(3.4744, grad_fn=<AddBackward0>)


Training :  98%|███████████████████████████████████████████████████████████████████▉ | 123/125 [00:15<00:00, 38.36it/s]

loss tensor(2.7963, grad_fn=<AddBackward0>)


Training : 100%|█████████████████████████████████████████████████████████████████████| 125/125 [00:16<00:00,  7.71it/s]


In [13]:
import imageio
images_trainer = []
obs = env.reset()
print(env.render_mode)
dones = np.zeros(env.num_envs, dtype=bool)
img = env.render()
active = np.ones(env.num_envs, dtype=bool)
while np.any(active):

    images_trainer.append(img)
    action, _ = dagger_trainer.policy.predict(obs)
    obs, reward, dones, info = env.step(action)
    img = env.render()

    dones &= active
    active &= ~dones
    
print(len(images_trainer))

rgb_array
1000


In [14]:
from datetime import datetime
imageio.mimsave(f'src/videos/dagger_trainer_{datetime.now().strftime("%d_%m_%Y_%H_%M")}.gif', images_trainer)

In [15]:
with tempfile.TemporaryDirectory(prefix="dagger_example_") as tmpdir:
    print(tmpdir)

NameError: name 'tempfile' is not defined